In [1]:
import torch
import numpy as np
import torch.nn.functional as F

from utils.utils import *
from datetime import datetime
from evaluation.train_inception_model import SpectrogramInception3
from data.preprocessing import AudioProcessor
from torch.utils.data import DataLoader
from evaluation.metrics.inception_score import InceptionScore
from evaluation.inception_models import DEFAULT_FOOTSTEPS_INCEPTION_MODEL
from data.audio_transforms import MelScale
from tqdm import trange
import ipdb

from os.path import dirname, realpath, join
import logging
from data.loaders import get_data_loader

In [2]:
config = {
    "model_name": "footsteps_inception_model_best_2021-04-29.pt",
    "comments": "inception trained on footsteps dataset",
    "state_dict_path": "evaluation/inception_models/footsteps_inception_model_best_2021-04-29.pt",
    
    "real_samples_path": "audio/footsteps_real",
    "synth_samples_path": "audio/footsteps_generated_audio_23-04-2021_15h",
    
    "output_path": "evaluation",
    "output_folder": "evaluation_metrics",
    
    "batch_size": 20,

    "real_samples_loader_config": {
        "dbname": "footsteps",
        "data_path": "audio/footsteps_real/",
        "criteria": {},
        "shuffle": True,
        "tr_val_split": 1.0
    },

    "synth_samples_loader_config": {
        "dbname": "footsteps",
        "data_path": "audio/footsteps_generated_audio_23-04-2021_15h/",
        "criteria": {},
        "shuffle": True,
        "tr_val_split": 1.0
    },
    
    "transform_config": {
        "transform": "stft",
        "fade_out": True,
        "fft_size": 1024,
        "win_size": 1024,
        "n_frames": 64,
        "hop_size": 256,
        "log": False,
        "ifreq": False,
        "sample_rate": 16000,
        "audio_length": 16000
    }
}

In [3]:
model_name = config['model_name']
state_dict_path = config['state_dict_path']
output_path = mkdir_in_path(config['output_path'], config['output_folder'])
# output_log = join(output_path, f"{model_name}_evaluation.log")
# logging.basicConfig(filename=output_log, level=logging.INFO)

In [4]:
real_samples_loader_config = config['real_samples_loader_config']

transform_config = config['transform_config']
transform = transform_config['transform']

dbname = real_samples_loader_config['dbname']

batch_size = config['batch_size']

processor = AudioProcessor(**transform_config)

loader_module = get_data_loader(dbname)

real_samples_loader = loader_module(name=dbname + '_' + transform, preprocessing=processor, **real_samples_loader_config)

n_real_samples = len(real_samples_loader)
print('n_real_samples: ', n_real_samples)

real_samples_data_loader = DataLoader(real_samples_loader,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=2)

device = 'cuda' if GPU_is_available() else 'cpu'

state_dict = torch.load(state_dict_path, map_location=device)
inception_footsteps = SpectrogramInception3(state_dict['fc.weight'].shape[0], aux_logits=False)
inception_footsteps.load_state_dict(state_dict)
inception_footsteps = inception_footsteps.to(device)

mel = MelScale(sample_rate=transform_config['sample_rate'],
                fft_size=transform_config['fft_size'],
                n_mel=transform_config.get('n_mel', 256),
                rm_dc=True)
mel = mel.to(device)

Configuring stft transform...
Dataset audio/footsteps_real/processed/footsteps_stft/footsteps_stft.pt exists. Reloading...
n_real_samples:  720
Cuda not available. Running on CPU


In [ ]:
is_maker_real_samples = InceptionScore()
inception_score_real_samples = []

with torch.no_grad():
    for batch_idx, data in enumerate(real_samples_data_loader):
        input, labels = data
        input.to(device)
        input = mel(input.float())
        mag_input = F.interpolate(input[:, 0:1], (299, 299))
        
        preds = inception_footsteps(mag_input.float())
        
        is_maker_real_samples.updateWithMiniBatch(preds)
        inception_score_real_samples.append(is_maker_real_samples.getScore())
        
        print('batch: ', batch_idx, 'IS: ', is_maker_real_samples.getScore())

In [ ]:
IS_mean = np.mean(inception_score_real_samples)
IS_std = np.std(inception_score_real_samples)
output_file = f'{output_path}/IS_real_{str(n_real_samples)}_{model_name}_{datetime.now().strftime("%d-%m-%y_%H_%M")}.txt'

with open(output_file, 'w') as f:
    f.write(str(IS_mean) + '\n')
    f.write(str(IS_std))
    f.close()

In [5]:
synth_samples_loader_config = config['synth_samples_loader_config']
synth_samples_loader = loader_module(name=dbname + '_' + transform, preprocessing=processor, **synth_samples_loader_config)

n_synth_samples = len(synth_samples_loader)
print('n_synth_samples: ', n_synth_samples)

synth_samples_data_loader = DataLoader(synth_samples_loader,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        num_workers=2)

Dataset audio/footsteps_generated_audio_23-04-2021_15h/processed/footsteps_stft/footsteps_stft.pt exists. Reloading...
n_synth_samples:  600


In [6]:
is_maker_synth_samples = InceptionScore()
inception_score_synth_samples = []

with torch.no_grad():
    for batch_idx, data in enumerate(synth_samples_data_loader):
        input, labels = data
        input.to(device)
        input = mel(input.float())
        mag_input = F.interpolate(input[:, 0:1], (299, 299))
        
        preds = inception_footsteps(mag_input.float())
        
        is_maker_synth_samples.updateWithMiniBatch(preds)
        inception_score_synth_samples.append(is_maker_synth_samples.getScore())
        
        print('batch: ', batch_idx, 'IS: ', is_maker_synth_samples.getScore())

batch:  0 IS:  4.085232043537821
batch:  1 IS:  4.674277961947189
batch:  2 IS:  4.654448693073675
batch:  3 IS:  4.805453150592395
batch:  4 IS:  4.948744928232177
batch:  5 IS:  4.849175483672129
batch:  6 IS:  4.90566440837025
batch:  7 IS:  4.912536781622127
batch:  8 IS:  4.8753353045734436
batch:  9 IS:  4.886369571312233
batch:  10 IS:  4.88847368398182
batch:  11 IS:  4.888763499295625
batch:  12 IS:  4.9031277668366915
batch:  13 IS:  4.887409523834624
batch:  14 IS:  4.895245391251997
batch:  15 IS:  4.912484390870489
batch:  16 IS:  4.9001870596698955
batch:  17 IS:  4.910064498141173
batch:  18 IS:  4.915399232834431
batch:  19 IS:  4.918477412783306
batch:  20 IS:  4.911068675264468
batch:  21 IS:  4.926493657398795
batch:  22 IS:  4.914464731317266
batch:  23 IS:  4.906985841311886
batch:  24 IS:  4.889293233477872
batch:  25 IS:  4.875577041907564
batch:  26 IS:  4.8644054662110845
batch:  27 IS:  4.857405671739421
batch:  28 IS:  4.878203754891297
batch:  29 IS:  4.8785

In [8]:
IS_mean = np.mean(inception_score_synth_samples)
IS_std = np.std(inception_score_synth_samples)
output_file = f'{output_path}/IS_synth_{str(n_synth_samples)}_{model_name}_{datetime.now().strftime("%d-%m-%y_%H_%M")}.txt'

with open(output_file, 'w') as f:
    f.write(str(IS_mean) + '\n')
    f.write(str(IS_std))
    f.close()